### DB creation and connection test

- create pollination DB 

In [1]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1"
)
mycur = mydb.cursor()

query = """
    create database if not exists pollination;
"""
mycur.execute(query)

query = """
show databases;
"""
mycur = mydb.cursor()
mycur.execute(query)
print(mycur.fetchall())

mydb.close()

[('information_schema',), ('mysql',), ('performance_schema',), ('pollination',), ('sys',)]


- create table

In [7]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1",
    database="pollination"
)
mycur = mydb.cursor()

query = """
    CREATE TABLE IF NOT EXISTS client_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        client_address VARCHAR(16),
        received_data int
    );
"""
mycur.execute(query)

query = """
    SHOW TABLES;
"""
mycur.execute(query)
tables = mycur.fetchall()
print(tables)

mydb.close()


[('client_data',)]


### Server client test

In [ ]:
import socket

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    port_num = 9988
    server_socket.bind(('192.168.0.13', port_num))
    server_socket.listen(5)
    print(f"server listening on port {port_num}")
    
    while True:
        try:
            client_socket, address = server_socket.accept()
            print(f"Connection from {address} has been established!")
            
            client_socket.sendall("Welcome to the server!".encode('utf-8'))
            
            while True:
                data = client_socket.recv(1024)
                if not data:
                    break
                
                # 받은 데이터를 출력
                print(f"Received : {data.decode()}")
                
                # 클라이언트로 "정상 입니다" 메시지를 전송
                if data.decode() == "0":
                    message = "정상 입니다".encode('utf-8')
                elif data.decode() == "1":
                    message = "비정상 입니다".encode('utf-8')
                client_socket.sendall(message)
                
            client_socket.close()
        except Exception as e:
            print(f"Error : {e}")
        
if __name__ == '__main__':
    start_server()


### 통합 코드 v1

In [1]:
import socket
import threading
import mysql.connector

# Global database connection and cursor
mydb = None
mycur = None

def connect_to_db():
    global mydb, mycur
    try:
        # Connect to the database (established once)
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="1",
            database="pollination"
        )
        mycur = mydb.cursor()
        print("Database connection established.")
    except mysql.connector.Error as err:
        print(f"Database connection error: {err}")

def close_db():
    global mydb, mycur
    if mycur:
        mycur.close()
    if mydb:
        mydb.close()
    print("Database connection closed.")

def insert_data_into_db(address, data):
    global mycur, mydb
    try:
        if mycur is None or mydb is None:
            print("Database connection is not established.")
            return
        
        # SQL query to insert data
        query = "INSERT INTO client_data (client_address, received_data) VALUES (%s, %s)"
        values = (address, data)
        
        # Execute the query and commit
        mycur.execute(query, values)
        mydb.commit()
        
        print(f"Data from {address} inserted into the database.")
        
    except mysql.connector.Error as err:
        print(f"Error inserting data: {err}")

def handle_client(client_socket, address):
    print(f"Connection from {address} has been established!")
    
    # 클라이언트에게 환영 메시지 전송
    # client_socket.sendall("Welcome to the server!".encode('utf-8'))
    
    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                break
            
            # 받은 데이터를 출력
            decoded_data = data.decode()
            print(f"Received from {address}: {decoded_data}")
            
            # Store the data into the database
            insert_data_into_db(address[0], decoded_data)
            
            # 클라이언트로 "정상 입니다" 또는 "비정상 입니다" 메시지를 전송
            if decoded_data == "0":
                message = "정상 입니다".encode('utf-8')
                client_socket.sendall(message)
            elif decoded_data == "1":
                message = "비정상 입니다".encode('utf-8')
                client_socket.sendall(message)
        except Exception as e:
            print(f"Error while handling client {address}: {e}")
            break

    client_socket.close()

def start_server(port_num):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('192.168.0.13', port_num))
    server_socket.listen(5)
    print(f"Server listening on port {port_num}")
    
    while True:
        client_socket, address = server_socket.accept()
        client_handler = threading.Thread(target=handle_client, args=(client_socket, address))
        client_handler.daemon = True  # 데몬 스레드 설정
        client_handler.start()

if __name__ == '__main__':
    # Connect to the database once
    connect_to_db()
    
    ports = [1234, 3141, 8888, 4040]
    
    # 각각의 포트에서 서버 시작
    for port in ports:
        server_thread = threading.Thread(target=start_server, args=(port,))
        server_thread.daemon = True  # 데몬 스레드 설정
        server_thread.start()

    try:
        # 주 스레드가 종료되지 않도록 대기
        while True:
            pass  # 또는 time.sleep()을 사용할 수 있습니다
    except KeyboardInterrupt:
        print("Shutting down server...")

    # Close the database connection when the server stops
    close_db()


Database connection established.
Server listening on port 1234
Server listening on port 3141
Server listening on port 8888
Server listening on port 4040
Connection from ('192.168.1.17', 40118) has been established!
Received from ('192.168.1.17', 40118): 1
Data from 192.168.1.17 inserted into the database.
Connection from ('192.168.1.17', 40128) has been established!
Received from ('192.168.1.17', 40128): 0
Data from 192.168.1.17 inserted into the database.
Connection from ('192.168.1.17', 40130) has been established!
Received from ('192.168.1.17', 40130): 2
Data from 192.168.1.17 inserted into the database.
Connection from ('192.168.1.17', 59368) has been established!
Received from ('192.168.1.17', 59368): 2
Data from 192.168.1.17 inserted into the database.
Connection from ('192.168.1.17', 35800) has been established!
Received from ('192.168.1.17', 35800): 1
Data from 192.168.1.17 inserted into the database.
Connection from ('192.168.1.17', 35804) has been established!
Received from (